In [ ]:
import pandas as pd
import stanza
import requests

# ==============================================================================
# VAIHE 0: ALUSTUKSET (kuten aiemmin)
# ==============================================================================



# Stop-sanojen lataus ja karsinta
url = "https://raw.githubusercontent.com/stopwords-iso/stopwords-fi/master/stopwords-fi.txt"
response = requests.get(url)
finnish_stop_words = response.text.splitlines()
suodatus = {'ei', 'eikä', 'mutta', 'vaan', 'vaikka', 'jos', 'kuin'}
custom_stop_words = [word for word in finnish_stop_words if word not in suodatus]
custom_stop_words.append('placeholder') #tyhjien otsikoiden tilalle laitettu teksti

# ==============================================================================
# VAIHE 1: DATAN LATAUS JA VALMISTELU
# ==============================================================================

# Ladataan vanha, manuaalisesti leimattu aineisto mallin koulutusta varten
print("Ladataan prototyypin opetusdataa (arvostelut.csv)...")
df_train_prototype = pd.read_csv('arvostelut.csv', delimiter=';', quoting=0)

# Ladataan uusi, suuri ja leimaamaton aineisto
print("Ladataan uutta, suurta aineistoa (yhdistetyt_arvostelut.csv)...")
df_new_reviews = pd.read_csv('yhdistetyt_arvostelut.csv')

# ==============================================================================
# VAIHE 2: TIEDOSTO 1 - OHJELMAN LUOMAT TUNNEARVOT
# ==============================================================================

from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

# Määritellään ja koulutetaan prototyyppimalli
print("Koulutetaan prototyyppimalli...")
X_train_proto = df_train_prototype['teksti']
y_train_proto = df_train_prototype['tunne']

text_clf = Pipeline([
    ('tfidf', TfidfVectorizer(stop_words=custom_stop_words)),
    ('clf', LogisticRegression(class_weight='balanced')),
])

text_clf.fit(X_train_proto, y_train_proto)
print("Prototyyppimalli koulutettu.")

# Tehdään ennusteet koko uudelle aineistolle
print("Tehdään ennusteita uudelle aineistolle...")
X_new = df_new_reviews['teksti']
predicted_sentiments = text_clf.predict(X_new)

# Lisätään ennusteet uuteen DataFrameen ja tallennetaan
df_model_predictions = df_new_reviews.copy()
df_model_predictions['mallin_ennuste'] = predicted_sentiments

output_filename_1 = 'mallin_luomat_tunnearvot3.csv'
df_model_predictions.to_csv(output_filename_1, index=False, encoding='utf-8-sig')
print(f"Tiedosto 1 tallennettu nimellä: {output_filename_1}")

# ==============================================================================
# VAIHE 3: TIEDOSTO 2 - TÄHTIEN PERUSTEELLA ANNETUT TUNNEARVOT
# ==============================================================================

def stars_to_sentiment(rating):
    """Muuttaa tähtiarvion (1-5) tunnesanaksi."""
    try:
        # Yritetään muuttaa arvo numeroksi. 
        rating = int(float(rating)) 
        if rating <= 2:
            return 'negatiivinen'
        elif rating == 3:
            return 'neutraali'
        elif rating >= 4:
            return 'positiivinen'
    except (ValueError, TypeError):
        # Jos muunnos epäonnistuu (esim. tekstiä tai tyhjä), palautetaan N/A
        return 'N/A'

print("Luodaan tunnearvoja tähtien perusteella...")
# Luodaan uusi DataFrame, johon lisätään tähtipohjaiset tunnearvot
df_star_sentiments = df_new_reviews.copy()
df_star_sentiments['tunne'] = df_star_sentiments['tahdet'].apply(stars_to_sentiment)

output_filename_2 = 'tahtiarvot3.csv'
df_star_sentiments.to_csv(output_filename_2, index=False, encoding='utf-8-sig')
print(f"Tiedosto 2 tallennettu nimellä: {output_filename_2}")

print("\nValmis! Voit nyt avata ja vertailla luotuja CSV-tiedostoja.")

Ladataan prototyypin opetusdataa (arvostelut.csv)...
Ladataan uutta, suurta aineistoa (yhdistetyt_arvostelut.csv)...
Koulutetaan prototyyppimalli...
Prototyyppimalli koulutettu.
Tehdään ennusteita uudelle aineistolle...
Tiedosto 1 tallennettu nimellä: mallin_luomat_tunnearvot3.csv
Luodaan tunnearvoja tähtien perusteella...
Tiedosto 2 tallennettu nimellä: tahtiarvot3.csv

Valmis! Voit nyt avata ja vertailla luotuja CSV-tiedostoja.
